## Load Data and Tokenizer

In [1]:
from transformers import BertTokenizer
from datasets import load_dataset

ds = load_dataset('openpecha/tagged_cleaned_MT_v1.0.3')

tokenizer = BertTokenizer.from_pretrained('tibetan_tokenizer')

ds['train'][0]

README.md:   0%|          | 0.00/479 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/172M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1429192 [00:00<?, ? examples/s]

{'Source': 'ཐུབ་པས་རྟག་ཏུ་དེ་བཞིན་སྤྱད།།',
 'Target': 'The aspirant should move in such a way at all times.',
 'File_Name': 'TM2382',
 'Machine Aligned': True,
 '__index_level_0__': 0,
 'Tag': 'Prophecies, Rituals'}

In [2]:
def condition(example):
    return example['Tag'] != ''

ds = ds.filter(condition)

Filter:   0%|          | 0/1429192 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Source', 'Target', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'],
        num_rows: 709531
    })
})

## Preprocess Data

In [4]:
all_tags = list(set(ds['train']['Tag']))

# Create a label-to-index mapping
label2id = {label: idx for idx, label in enumerate(all_tags)}
id2label = {idx: label for label, idx in label2id.items()}

# Save label mappings for future use
import json
with open("op_label_mapping.json", "w") as f:
    json.dump(label2id, f)


In [5]:
def preprocess(examples):
    tokens = tokenizer(examples["Source"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] = [label2id[label] for label in examples["Tag"]]    
    return tokens

encoded_dataset = ds.map(preprocess, batched=True)


Map:   0%|          | 0/709531 [00:00<?, ? examples/s]

In [6]:
encoded_dataset['train'][0]

{'Source': 'ཐུབ་པས་རྟག་ཏུ་དེ་བཞིན་སྤྱད།།',
 'Target': 'The aspirant should move in such a way at all times.',
 'File_Name': 'TM2382',
 'Machine Aligned': True,
 '__index_level_0__': 0,
 'Tag': 'Prophecies, Rituals',
 'input_ids': [2,
  2820,
  9,
  339,
  9,
  1918,
  9,
  40,
  9,
  42,
  9,
  3925,
  9,
  1321,
  11,
  11,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [7]:
encoded_dataset = encoded_dataset.remove_columns(['Source', 'Target', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'])

In [8]:
encoded_dataset = encoded_dataset['train'].train_test_split(.15)

## Train Model

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load tokenizer and model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id))

# Resize embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU
model = model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    
    # Get predicted class indices
    predictions = np.argmax(predictions, axis=1)
    
    # Compute metrics
    accuracy = accuracy_score(references, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(references, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


In [11]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir="op-bert-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # Set a maximum number of epochs
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",  # Save the model at the end of every epoch
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="accuracy",  # Metric to monitor
    greater_is_better=True,  # Higher accuracy is better
    logging_dir="./logs"
)

# Add the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if the metric does not improve for 3 evaluation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Start training
trainer.train()

/tmp/ipykernel_22135/739678573.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/3769400 [00:00<?, ?it/s]

{'loss': 2.0894, 'grad_norm': 3.4238860607147217, 'learning_rate': 1.9997347057887198e-05, 'epoch': 0.01}
{'loss': 2.0178, 'grad_norm': 3.957597017288208, 'learning_rate': 1.9994694115774395e-05, 'epoch': 0.03}
{'loss': 1.9869, 'grad_norm': 7.211489677429199, 'learning_rate': 1.999204117366159e-05, 'epoch': 0.04}
{'loss': 1.9831, 'grad_norm': 4.002837657928467, 'learning_rate': 1.9989388231548788e-05, 'epoch': 0.05}
{'loss': 1.9785, 'grad_norm': 5.943065643310547, 'learning_rate': 1.9986735289435985e-05, 'epoch': 0.07}
{'loss': 1.9586, 'grad_norm': 4.799134254455566, 'learning_rate': 1.9984082347323185e-05, 'epoch': 0.08}
{'loss': 1.9361, 'grad_norm': 4.495475769042969, 'learning_rate': 1.9981429405210378e-05, 'epoch': 0.09}
{'loss': 1.9416, 'grad_norm': 4.601923942565918, 'learning_rate': 1.9978776463097578e-05, 'epoch': 0.11}
{'loss': 1.9231, 'grad_norm': 5.0125932693481445, 'learning_rate': 1.9976123520984775e-05, 'epoch': 0.12}
{'loss': 1.9063, 'grad_norm': 5.557928085327148, 'lear

  0%|          | 0/6652 [00:00<?, ?it/s]

/home/j/Desktop/MLotsawa/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

{'eval_loss': 1.592553734779358, 'eval_accuracy': 0.495950389927652, 'eval_f1': 0.43995122769298595, 'eval_precision': 0.4455531113799034, 'eval_recall': 0.495950389927652, 'eval_runtime': 245.9537, 'eval_samples_per_second': 432.724, 'eval_steps_per_second': 27.046, 'epoch': 10.0}
{'loss': 1.2609, 'grad_norm': 20.011045455932617, 'learning_rate': 1.7999681646946466e-05, 'epoch': 10.0}
{'loss': 1.1717, 'grad_norm': 22.799793243408203, 'learning_rate': 1.799702870483366e-05, 'epoch': 10.01}
{'loss': 1.188, 'grad_norm': 13.32534122467041, 'learning_rate': 1.799437576272086e-05, 'epoch': 10.03}
{'loss': 1.1677, 'grad_norm': 20.454381942749023, 'learning_rate': 1.7991722820608056e-05, 'epoch': 10.04}
{'loss': 1.2074, 'grad_norm': 27.273958206176758, 'learning_rate': 1.7989069878495253e-05, 'epoch': 10.05}
{'loss': 1.1823, 'grad_norm': 22.1530818939209, 'learning_rate': 1.798641693638245e-05, 'epoch': 10.07}
{'loss': 1.1558, 'grad_norm': 26.75925064086914, 'learning_rate': 1.798376399426964

  0%|          | 0/6652 [00:00<?, ?it/s]

/home/j/Desktop/MLotsawa/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.6183372735977173, 'eval_accuracy': 0.49539603495255097, 'eval_f1': 0.4422287990438464, 'eval_precision': 0.4446891482403874, 'eval_recall': 0.49539603495255097, 'eval_runtime': 246.0357, 'eval_samples_per_second': 432.579, 'eval_steps_per_second': 27.037, 'epoch': 11.0}
{'train_runtime': 50323.5068, 'train_samples_per_second': 1198.448, 'train_steps_per_second': 74.903, 'train_loss': 1.4578601861428213, 'epoch': 11.0}


TrainOutput(global_step=414634, training_loss=1.4578601861428213, metrics={'train_runtime': 50323.5068, 'train_samples_per_second': 1198.448, 'train_steps_per_second': 74.903, 'total_flos': 4.364592657569042e+17, 'train_loss': 1.4578601861428213, 'epoch': 11.0})